# Algorithms Part I, Week I Programming Assignment

## Percolation

### Problem Statement

#### Percolation.

Given a composite systems comprised of randomly distributed insulating and metallic materials: what fraction of the materials need to be metallic so that the composite system is an electrical conductor? Given a porous landscape with water on the surface (or oil below), under what conditions will the water be able to drain through to the bottom (or the oil to gush through to the surface)? Scientists have defined an abstract process known as percolation to model such situations.

#### The model.

We model a percolation system using an n-by-n grid of sites. Each site is either open or blocked. A full site is an open site that can be connected to an open site in the top row via a chain of neighboring (left, right, up, down) open sites. We say the system percolates if there is a full site in the bottom row. In other words, a system percolates if we fill all open sites connected to the top row and that process fills some open site on the bottom row. (For the insulating/metallic materials example, the open sites correspond to metallic materials, so that a system that percolates has a metallic path from top to bottom, with full sites conducting. For the porous substance example, the open sites correspond to empty space through which water might flow, so that a system that percolates lets water fill open sites, flowing from top to bottom.)

#### The problem. 

In a famous scientific problem, researchers are interested in the following question: if sites are independently set to be open with probability p (and therefore blocked with probability 1 − p), what is the probability that the system percolates? When p equals 0, the system does not percolate; when p equals 1, the system percolates. When n is sufficiently large, there is a threshold value p* such that when p < p* a random n-by-n grid almost never percolates, and when p > p*, a random n-by-n grid almost always percolates. No mathematical solution for determining the percolation threshold p* has yet been derived. Your task is to write a computer program to estimate p*.

### Step 0: Import Modules for Use in Solution

In [245]:
import math
import random
import statistics

### Step 1: Define the Site Class

The Site class includes the following functions:

#### __init__

Initializes a site.  Is called from the Grid __init__ class and takes as input the [row, column] location of the site, along with the overall size of the grid.
    
    status - Boolean - True if site is open.  The site in Row 0 and the site in Row N + 1 start out as True, all other start out as False

    size - Integer - All sites start out as size = 1
    
    location - The [row, column] location of the site in the grid matrix.  This does not change.

    root - The root of the site.  Initially, this is the site itself.

#### is_open

Returns a boolean as to the state of status

#### open_site

Changes the boolean status of a site to True

#### __str__

Formats the site information when printed

In [279]:
class Site:
    
    def __init__(self, i, j, N):
        self.status = True if i == 0 or i == N + 1 else False
        self.size = 1
        self.location = [i,j]
        self.root = self
        
    def is_open(self):
        return self.status
    
    def open_site(self):
        self.status = True
        
    def __str__(self):
        return "<status = " + str(self.status) + " , size = " + str(self.size) + ", location = " + str(self.location) + ", root = " + str(self.root.location) + ">"
        

Test functionality of the Site Class

In [280]:
site = Site(1,1,2)
print(site)
print(site.is_open())
site.open_site()
print(site.is_open())
print(site)

<status = False , size = 1, location = [1, 1], root = [1, 1]>
False
True
<status = True , size = 1, location = [1, 1], root = [1, 1]>


### Step 2: Define the Grid Class

The Grid class sets up the grid for percolation.  The grid class includes the following functions:

#### __init__

Initializes a grid of (N+2) x (N+2) sites.  The top and bottom rows have a single site, the middle rows have N+2 sites.  The extra columns on right and left simplify checking for adjacent squares.  Initializes the number of open_sites to zero.

#### get_site

Given a pair of numbers, returns the site associated with the location.

#### get_random_site

No inputs. Returns a random previously unopened site.

#### open_site

Given a site, performs the following functions:

    Increases the number of open_sites
    
    Opens the site
    
    Calls get neighbors function
    
    Calls the Union function if neighbor is open and not already connected to the neighbor
    
#### get_neighbors

Given a site, returns its neighbors.  The top row and bottom row are neighbors to all sites below and above them, respectively.

#### find_root

Given a site, returns its ultimate root and sets the root and its parent's root equal to its root.

#### is_connected

Given two sites, returns True if they are connected.

#### union

Given two sites, sets the root of the smaller tree to the root of the larger tree and updates the size of the tree.  Does not return anything.

#### is_percolating

No inputs.  Returns True if the top row is connected to the bottom row.

#### __str__

Formats the grid information when printed.


In [291]:
import random

class Grid:
    
    def __init__(self,N):
        self.grid = []
        for i in range(N+2):
            row = []
            if i == 0 or i == N + 1:
                row.append(Site(i,0,N))
            else:
                for j in range(N+2):
                    site = Site(i,j,N)
                    row.append(site)
            self.grid.append(row)
        self.open_sites = 0
    
    def get_site(self,i,j):
        if i == 0 or i == N + 1:
            return self.grid[i][0]
        return self.grid[i][j]
    
    def get_random_site(self):
        while True:
            site = self.get_site(random.choice(range(1,N+1)), random.choice(range(1,N+1)))
            if not site.is_open():
                return site
    
    def open_site(self, site):
        self.open_sites += 1
        site.open_site()
        neighbors = self.get_neighbors(site)
        for neighbor in neighbors:
            if neighbor.is_open() and not self.is_connected(site, neighbor):
                self.union(site, neighbor)
    
    def get_neighbors(self, site):
        i, j = site.location
        top = self.grid[0][0] if i == 1 else self.grid[i-1][j]
        left = self.grid[i][j-1]
        right = self.grid[i][j+1]
        bottom = self.grid[i + 1][0] if i == N else self.grid[i+1][j]
        return [top, left, right, bottom]
    
    def find_root(self,p):
        while p != p.root:
            p.root = p.root.root
            p = p.root
        return p
    
    def is_connected(self, p, q):
        return self.find_root(p) == self.find_root(q)
        
    def union(self, p, q):
        root_p = self.find_root(p)
        root_q = self.find_root(q)   
        if root_p.size < root_q.size:           
            root_p.root = root_q.root
        else:
            root_q.root = root_p.root
        root_p.size = root_q.size = root_p.size + root_q.size

    def is_percolating(self):
        return self.is_connected(self.get_site(0,0),self.get_site(N+1,0))
    
    def get_open_sites(self):
        return self.open_sites
    
    def __str__(self):
        return str([[str(site) for site in row] for row in self.grid])
        

Test functionality of the grid class

In [294]:
N = 3
grid = Grid(N)
site = grid.get_site(1,2)
print(site)
random_site = grid.get_random_site()
print(random_site)
print('-----------------------')
print(grid)
print('-----------------------')

site_1 = grid.get_site(1,2)
site_2 = grid.get_site(3,3)
site_3 = grid.get_site(2,2)
site_4 = grid.get_site(3,2)

grid.open_site(site_1)

grid.open_site(site_2)
grid.open_site(site_3)
print(grid.is_percolating())

grid.open_site(site_4)

print(grid.is_percolating())
print(grid.get_open_sites())


<status = False , size = 1, location = [1, 2], root = [1, 2]>
<status = False , size = 1, location = [2, 3], root = [2, 3]>
-----------------------
[['<status = True , size = 1, location = [0, 0], root = [0, 0]>'], ['<status = False , size = 1, location = [1, 0], root = [1, 0]>', '<status = False , size = 1, location = [1, 1], root = [1, 1]>', '<status = False , size = 1, location = [1, 2], root = [1, 2]>', '<status = False , size = 1, location = [1, 3], root = [1, 3]>', '<status = False , size = 1, location = [1, 4], root = [1, 4]>'], ['<status = False , size = 1, location = [2, 0], root = [2, 0]>', '<status = False , size = 1, location = [2, 1], root = [2, 1]>', '<status = False , size = 1, location = [2, 2], root = [2, 2]>', '<status = False , size = 1, location = [2, 3], root = [2, 3]>', '<status = False , size = 1, location = [2, 4], root = [2, 4]>'], ['<status = False , size = 1, location = [3, 0], root = [3, 0]>', '<status = False , size = 1, location = [3, 1], root = [3, 1]>', 

### Step 3: Define the Function for one run of percolation

Takes the size of the grid as input.  Initializes the grid. Opens sites until the grid percolates. Returns number of open sites.

In [300]:
def percolate(N):
    grid = Grid(N)
    while not grid.is_percolating():
        grid.open_site(grid.get_random_site())
    return grid.get_open_sites()

Test functionality of percolate

In [303]:
N = 4
t = 3
for _ in range(t):
    print(percolate(N))

11
10
10


### Step 4: Define Monte Carlo Function

Takes size of grid (N) and number of iterations(t) as inputs and returns of list of the number of open cells needed to percolate on each iteration.

In [305]:
def monte_carlo(N,t):
    return [percolate(N) for _ in range(t)]

Test functionality of monte_carlo

In [306]:
N = 4
t = 5
print(monte_carlo(N,t))

[9, 9, 10, 10, 8]


### Step 5: Compile statistics

Takes the list from monte_carlo (openings), size(N) and iterations(t) as input and returns the mean, standard deviation, and 95% confidence intervals.

In [307]:
def compile_statistics(openings, N, t):
    percentage_open = [number_open / N**2 for number_open in openings]
    mean = statistics.mean(percentage_open)
    standard_deviation = statistics.stdev(percentage_open)
    confidence_interval = [mean - 1.96 * standard_deviation / math.sqrt(t), mean + 1.96 * standard_deviation / math.sqrt(t) ]
    return mean, standard_deviation, confidence_interval

### Step 6: Put It All Together

Run size 10, 20 and 100 sizes with 20, 100, and 1000 iterations.

In [310]:
sizes = [10, 20, 100]
iterations = [20, 100, 1000]

for N in sizes:
    for t in iterations:
        simulation_results = monte_carlo(N,t)
        mean, standard_deviation, confidence_interval = compile_statistics(simulation_results, N, t)
        print('Size = ', N)
        print('Iterations = ', t)
        print('Mean = ', mean)
        print('Standard Deviation = ', standard_deviation)
        print('Confidence Interval = ', confidence_interval)

Size =  10
Iterations =  20
Mean =  0.5875
Standard Deviation =  0.0449414824199788
Confidence Interval =  [0.5678035268986659, 0.6071964731013342]
Size =  10
Iterations =  100
Mean =  0.5901
Standard Deviation =  0.07789522877985959
Confidence Interval =  [0.5748325351591475, 0.6053674648408525]
Size =  10
Iterations =  1000
Mean =  0.59292
Standard Deviation =  0.07476806714008236
Confidence Interval =  [0.5882858271871434, 0.5975541728128566]
Size =  20
Iterations =  20
Mean =  0.59325
Standard Deviation =  0.04976986511619467
Confidence Interval =  [0.5714373948803619, 0.6150626051196382]
Size =  20
Iterations =  100
Mean =  0.596775
Standard Deviation =  0.048826241857626046
Confidence Interval =  [0.5872050565959054, 0.6063449434040947]
Size =  20
Iterations =  1000
Mean =  0.5902875
Standard Deviation =  0.04986529495943438
Confidence Interval =  [0.5871968169979486, 0.5933781830020514]
Size =  100
Iterations =  20
Mean =  0.590115
Standard Deviation =  0.017175082164822986
Conf